In [2]:
import pandas as pd
import numpy as np
from nibabel import freesurfer as fs
import networkx as nx
import brainx as bx
from brainx import weighted_modularity as wmod
from brainx.nodal_roles import (within_community_degree,
                               participation_coefficient)
from brainx.metrics import (weighted_degree)
from brainx.weighted_modularity import LouvainCommunityDetection
import os
import re
import copy
import time
import cPickle as pkl
from joblib import Parallel,delayed
import warnings
idx=pd.IndexSlice

/Applications/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Applications/anaconda/lib/python2.7/site-packages/brainx/__init__.py:30: UserWarning: Monkeypatching NetworkX's Watts-Strogatz routine
  warnings.warn("Monkeypatching NetworkX's Watts-Strogatz routine")


In [5]:
class Edge:
    '''create functional connectivity edge series with node and edge indices 
    roottxt : suffix of text file str
    (e.g., '.Schaefer2018_{}'+'Parcels_7Networks_{}')
    numparc : number of parcellations in Schaefer annot to use (e.g., '200')
    sub : sub folder string (e.g., 'ACT01_1')
    funcdir : directory structure str to have specific vars added
    ftype : type of functional data collected (e.g., 'rest') 
    run : run folder within subject and task (e.g., '001' or 'average')
    mscrub : scrubbing parameter (e.g., '0.5')
    nuis : str folder name of nuisance variables (compcor_ncomponents_5...)
    band : str folder name (e.g., '0.009.0.1')
    '''
    def __init__(self,sub,adjcsvdir,
                 community_sets=None,community_key=None,
                 annotpwd=None,fsavg='fsaverage5',numparc='200'):
        '''starts by importing a correlation matrix and setting other
        variables'''
        self.community_sets=community_sets
        self.fsavg=fsavg
        self.numparc=numparc
        self.sub=sub
        self.threshold=None
        self.cost=None
        self.adj_csvdir=adjcsvdir
        self.adj_df=pd.read_csv(adjcsvdir)
        self.numnodes=len(self.adj_df)
        self.adj_df.index=pd.Index(range(self.numnodes),name='parc_y')
        self.adj_df.columns=pd.Index(range(self.numnodes),name='parc_x')
        self.nodes=range(self.numnodes)
    
    def format_adj_df(self,x_node_names=['parc_x']):
        '''maintain info in only one diagonal of adj matrix and stacks
        which returns a pd.Dataframe of edge weights with each node identified
        
        x_node_names : column names to stack (ie melt)the adj matrix by.
        if you have already added communities, you'll want the default option,
        if no communities, then [parc_x] only.
        - note: stacking automatically removes na vals 
        (created from the np.tril_indices) 
        '''
        adj_df=self.adj_df
        adj_df.values[np.tril_indices_from(adj_df)]=np.nan
        self.edges_series=adj_df.stack(x_node_names)
        self.edges_series.name='weight'
    
    def normalize(self):
        '''use brainx util to make edges all positive'''
        norm_series=bx.util.normalize(self.edges_series)
        self.edges_series=norm_series
        
    def find_thresh_edges(self,cost):
        '''find the threshold correlation associated with
        cost (density) of edges to use.
        '''
        es=self.edges_series
        tot_edges=len(es)
        num_edges_tokeep=cost*tot_edges
        #may want to make more like brainx???
        num_edges_tokeep_int=np.int(num_edges_tokeep)
        es.sort_values(ascending=False,inplace=True)
        threshold=es.iloc[num_edges_tokeep_int]
        self.cost=cost
        self.threshold=threshold
        
    def make_nxGraph(self,binary=False):
        '''make a networkx graph with formatted edges
        loops through nodes (as defined by how I defined
        nodes in adjacency matrix in init).
        loops through the edges_series object containing
        the edges (containing only filtered if thresholded)
        '''
        es=self.edges_series
        if self.threshold:
            es_filt=bx.util.normalize(es[es>self.threshold])
            edges_df=es_filt.reset_index()
        else:
            edges_df=es.reset_index()
        if binary:
            edges_df['weight']=1
        graph=nx.Graph()
        self.graph=graph
        #loop through nodes and edges
        for node in self.nodes:
            graph.add_node(node)
        for group,edge in edges_df.iterrows():
            graph.add_edge(u=edge['parc_x'],
                           v=edge['parc_y'],
                           weight=edge['weight'])
    
    def add_GraphPartition(self,runLouvain=True):
        '''make a Graph Partition object
        that is used to calculate metrics 
        in brainx. Uses weighted partition object 
        stuff (add GraphPartition if you want to 
        use brainx metrics using GraphPartition)
        '''
        if runLouvain:
            louvain=LouvainCommunityDetection(graph=self.graph,
                                              communities=self.community_sets)
            partitions=louvain.run()
            self.graphpartition=partitions[-1]
        else:
            self.graphpartition=wmod.WeightedPartition(
                self.graph,self.community_sets)

In [6]:
###node-wise statistics
'''
See The modular and integrative functional architecture of the human brain
Bertolero , Yeo , and D'Esposito
'''
def calc_node_participation_coeff(graphpart):
    '''Participation coefficient is a measure of diversity of 
    intermodular connections of individual nodes.
    
    The function is from brainx and uses the weights from the
    networkx graph (within the GraphPartition object). 
    
    Thus, if binary weights desired (set weights of edges all to 1)
    '''
    node_part_coef=participation_coefficient(
        graphpart,
        edgeless =np.nan,
        catch_edgeless_node=False)
    return pd.Series(node_part_coef,name='p_coef')

def calc_node_within_community_degree(graphpart):
    '''Within community degree z score
    '''
    node_within_degree=within_community_degree(
        graphpart,
        edgeless=np.nan,
        catch_edgeless_node=False)
    return pd.Series(node_within_degree,name='node_within_degree')

def calc_node_degree(graph):
    '''Node degree is the number of links connected to the node
    Binarized matrices (for weighted, this is strength)
    
    If binary desired, set weights to 1.
    '''
    node_degree=weighted_degree(graph)
    return pd.Series(node_degree,name='node_degree')

def calc_modularity(graphpart):
    modularity=graphpart.modularity()
    return {'modularity':modularity}

def get_communities(graphpart):
    community_sets=enumerate(graphpart.communities)
    return pd.Series(
        {node:i for i,setlist in community_sets
         for node in setlist},
        name='communities')

In [7]:
#not using......
def init_Edges(sub,ftype='rest',mscrub='0.5',nuis_global='global1',
              get_community=None):
    '''initiate the edge object and add 7 network Yeo community sets based on
    a network name string key: network index integer values
    '''
    subint=int(re.findall('\d+',sub)[0])
    edge=Edge(
        sub=subint,
        adjcsvdir=funcpath+
        sub+'/'+ftype+'/average/_threshold_'+mscrub+'/'
        '_compcor_ncomponents_5_selector_pc10.'
        'linear1.wm1.'+nuis_global+'.motion1.quadratic1.gm0.compcor0.csf1/'
        '_bandpass_freqs_0.009.0.1/'
        'bh.Schaefer2018_400Parcels_7Networks_fsaverage5_zcorr.csv')
    #add community indices and sets
    if get_community:
        edge.add_community_sets(annotpwd=annotpwd,
                                community_key=network7_key)
        edge.format_adj_df(['parc_x','net_x'])
    else:
        edge.community_sets=None
        edge.format_adj_df()
    return edge

In [8]:
def run_Edges(edgeobj,cost=None):
    '''make a pd.Series of unique edges in adj_df
    if reusing an edge object, it makes new Graph/GraphPartition.
    objects based on threshold
    '''
    binary=None
    edgeobj.threshold=None
    edgeobj.cost=None
    if cost:
        edgeobj.find_thresh_edges(cost)
        #binary=True
    #normalize edges (no negative edges allowed in brainx)
    edgeobj.make_nxGraph(binary=binary)
    edgeobj.add_GraphPartition(runLouvain=True)

In [9]:
def run_Metrics(edgeobj,cost):
    '''updates an Edge object based on
    a threshold (with the run_Edge fxn)
    and then computes node-wise graph metrics
    from the brainx GraphPartition obj, the 
    graph, or the pd.Series.
    '''
    run_Edges(edgeobj,cost)
    sub=edgeobj.sub
    graph=edgeobj.graph
    graphpart=edgeobj.graphpartition
    edgeseries=edgeobj.edges_series
    cost=edgeobj.cost
    thresh=edgeobj.threshold
    communities=graphpart.communities
    
    node_metrics_df=(pd.DataFrame([
        calc_node_participation_coeff(graphpart),
        calc_node_within_community_degree(graphpart),
        get_communities(graphpart)])
                     .T.rename_axis(['parc_label']))
    
    index_columns={'subs':sub,'cost':cost,'threshold':thresh,
                  'num_communities':len(communities)}
    other_sub_metrics=calc_modularity(graphpart)
    sub_metrics_df=(pd.DataFrame(node_metrics_df.mean(axis=0))
                    .T       
                    .assign(**other_sub_metrics)
                    .assign(**index_columns))
    
    return (node_metrics_df.assign(**index_columns),
            sub_metrics_df,
           edgeobj)

In [10]:
def concat_Metrics(sub,cost_List,n_jobs=1):
    '''runs the run_Metrics for a 
    list of subs after formatting Edge object'''
    subedges=init_Edges(sub=sub)
    metrics_tuple=Parallel(n_jobs)(
        delayed(run_Metrics)
        (copy.deepcopy(subedges),cost)
        for cost in cost_List)
    node_metrics=pd.concat(x[0] for x in metrics_tuple)
    sub_metrics=pd.concat(x[1] for x in metrics_tuple)
    sub_edgeobjs={x[2].cost:x[2] for x in metrics_tuple}
    return (node_metrics,sub_metrics,sub_edgeobjs)

In [103]:
def create_subs_runList(ftype,datapwd):
    '''create a list of subject folders to
     include (these strs are used for input into
     the init_Edge fxn for initiating metric calculations
     with Edge class obj.)'''
    FDthresh={'memory':'50','rest':'10'}
    maxdisp={'memory':'4','rest':'50'}
    md=maxdisp[ftype]
    FDt=FDthresh[ftype]
    power_df=pd.read_csv(datapwd+ftype+'/power_params_allsubs.csv')
    motion_df=pd.read_csv(datapwd+ftype+'/motion_params_allsubs.csv')
    subs_runs=(motion_df
               .merge(power_df,on=['Subject','Scan'])
               .query('Max_Abs_Maxdisp<{} &'
                      'PercentFDgreaterthan050<{}'.format(md,FDt)))
    subs_runs['run']=subs_runs.Scan.str.extractall('(\d\d\d)').values
    #create subs and good runs groupby object iterator
    subs_run_groupby=(subs_runs
                  .groupby(['Subject'])
                  .apply(lambda x: x.run.values)
                  .groupby(level=0))
    subs_List=[sub for sub,runList in subs_run_groupby]
    return subs_List

In [104]:
subs_List=create_subs_runList('rest',('/Volumes/Users/mbkranz/projects/'
                            'git_repo_backups/'
                            'individual_diff_func_conn_corticalthick/data/'))

In [85]:
funcpath='/Volumes/Users/mbkranz/projects/ACT_Freesurfer_Func/'
gitpath=('/Volumes/Users/mbkranz/projects/git_repo_backups/'
          'individual_diff_func_conn_corticalthick/')
annotpwd=(gitpath+
          'parcellations/schaefer2018/FreeSurfer5.3/'
          'fsaverage5/label/'
          '{}.Schaefer2018_400Parcels_7Networks_order.annot')
adjcsv_name='bh.Schaefer2018_400Parcels_7Networks_fsaverage5_zcorr.csv'
hemi_key={'lh':0,'rh':1}
network7_key={n:i for i,n in enumerate(
    ['Medial_Wall','Vis','SomMot','DorsAttn',
     'SalVentAttn','Limbic','Cont','Default'])}
community_key={}
cost_List=list(np.arange(.05,.2,.01))
ftype='rest'
mscrub='0.5'
nuis_global='global1'

In [24]:
warnings.filterwarnings("ignore")

In [35]:
#run and save node metrics with Louvain community detection
for sub in subs_List:
    starttime=time.time()
    funcdir=(funcpath+sub+'/'+ftype+'/average/_threshold_'+mscrub+'/'
    '_compcor_ncomponents_5_selector_pc10.'
    'linear1.wm1.'+nuis_global+'.motion1.quadratic1.gm0.compcor0.csf1/'
    '_bandpass_freqs_0.009.0.1/')
    
    subint=int(re.findall('\d+',sub)[0])
    edge=Edge(sub=subint,adjcsvdir=funcdir+adjcsv_name)
    edge.community_sets=None
    edge.format_adj_df()
    node_mets,sub_mets,edgeobjs=concat_Metrics(sub,cost_List,n_jobs=16)
    node_mets.to_csv(funcdir+'node_metrics_Louvain.csv')
    sub_mets.to_csv(funcdir+'sub_metrics_Louvain.csv')
    f=open(funcdir+'edge_object_Louvain.pkl','wb')
    pkl.dump(edgeobjs,f)
    f.close()
    print('Done with '+sub+'Time: '+str(time.time()-starttime))

Done with ACT08_1Time: 587.360948086
Done with ACT09_1Time: 580.400264978
Done with ACT11_1Time: 553.179049015
Done with ACT12_1Time: 519.946748018
Done with ACT13_1Time: 557.218665123
Done with ACT16_1Time: 560.313765049
Done with ACT18_1Time: 576.597196102
Done with ACT20_1Time: 598.121393919
Done with ACT21_1Time: 694.80591917
Done with ACT22_1Time: 627.807766914
Done with ACT23_1Time: 536.605708122
Done with ACT24_1Time: 738.382025957
Done with ACT25_1Time: 578.761958838
Done with ACT6001_1Time: 548.20156002
Done with ACT6002_1Time: 495.37273407
Done with ACT6004_1Time: 626.953985929
Done with ACT6005_1Time: 533.547255993
Done with ACT6006_1Time: 563.72242403
Done with ACT6009_1Time: 556.806764126
Done with ACT6011_1Time: 604.169834852
Done with ACT6014_1Time: 476.363862038
Done with ACT6015_1Time: 486.242815018
Done with ACT6016_1Time: 494.463088989
Done with ACT6017_1Time: 632.533671141
Done with ACT6018_1Time: 616.746487856
Done with ACT6019_1Time: 563.55942297
Done with ACT6020

# unthresholded metrics (but from community structure based on Louvain comm detection)

In [152]:
def merge_edges_communities(edge_df,comms):
    node_comms=pd.DataFrame([[comm,node] 
                             for comm,node_set in enumerate(comms) 
                             for node in node_set],
                            columns=['community','parc'])
    edge_comms_df=(edge_df.reset_index()
                   .merge(right=node_comms,left_on='parc_x',right_on='parc',how='left')
                   .drop(['parc'],axis=1)
                   .merge(right=node_comms,left_on='parc_y',right_on='parc',how='left')
                   .drop(['parc'],axis=1))
    return edge_comms_df

def _compute_unthresh_stats(df,node):
    '''compute mean within network and between network average connectivity
    for an individual node from the edge series with a community index
    '''
    edge_node_df=df.query('parc_x=={} | parc_y=={}'
                                  .format(str(node),str(node)))
    within_str='community_x==community_y'
    btw_str='community_x!=community_y'
    within=edge_node_df.query(within_str)['weight'].mean()
    between=edge_node_df.query(btw_str)['weight'].mean()
    return {'parc_label':node,
            'within_strength':within,
            'between_strength':between}

def calc_unthresh_metrics(edge_df_wcomms,sub,cost):
    '''
    - for each node calc between sys, within sys strength
    with an edge series of all edges and community indices
    - label outputted df with a cost (if applicable) and subject
    '''
    node_copies=(edge_df_wcomms
                 .filter(['parc_x','parc_y'])
                 .values.flatten())
    nodes=np.unique(node_copies)
    stats=[_compute_unthresh_stats(edge_df_wcomms,node) for node in nodes]
    return pd.DataFrame(stats).assign(subs=sub,cost=cost)

def save_unthresh_metrics(sub):
    #load dict containing Edge objects (key is threshold,item is Edge object)
    funcdir=(funcpath+sub+'/'+ftype+'/average/_threshold_'+mscrub+'/'
         '_compcor_ncomponents_5_selector_pc10.'
         'linear1.wm1.'+nuis_global+'.motion1.quadratic1.gm0.compcor0.csf1/'
         '_bandpass_freqs_0.009.0.1/')
    edgeobjs=pkl.load(open(funcdir+'edge_object_Louvain.pkl'))
    #make a list of dicts with key,items representing input into 
    #calc_unthresh_metrics (using splat operator)
    edge_dict_list=[{
            'sub':int(re.findall('\d+',sub)[0]),
            'cost':cost,
            'edge_df_wcomms':merge_edges_communities(
                edgeobj.edges_series,edgeobj.graphpartition.communities)
            } for cost,edgeobj in edgeobjs.iteritems()]
    #get list of dataframes with metrics
    edge_metrics=[calc_unthresh_metrics(**edge_dict) 
                  for edge_dict in edge_dict_list]
    pd.concat(edge_metrics).to_csv(funcdir+'node_metrics_Louvain_unthresh.csv')
    return sub

In [118]:
Parallel(8)(delayed(save_unthresh_metrics)(
    sub) for sub in subs_List)

['ACT01_1',
 'ACT04_1',
 'ACT05_1',
 'ACT07_1',
 'ACT08_1',
 'ACT09_1',
 'ACT11_1',
 'ACT12_1',
 'ACT13_1',
 'ACT16_1',
 'ACT18_1',
 'ACT20_1',
 'ACT21_1',
 'ACT22_1',
 'ACT23_1',
 'ACT24_1',
 'ACT25_1',
 'ACT6001_1',
 'ACT6002_1',
 'ACT6004_1',
 'ACT6005_1',
 'ACT6006_1',
 'ACT6009_1',
 'ACT6011_1',
 'ACT6014_1',
 'ACT6015_1',
 'ACT6016_1',
 'ACT6017_1',
 'ACT6018_1',
 'ACT6019_1',
 'ACT6020_1',
 'ACT6021_1',
 'ACT6022_1',
 'ACT6024_1',
 'ACT6025_1',
 'ACT6026_1',
 'ACT6027_1',
 'ACT6028_1',
 'ACT6029_1',
 'ACT6030_1',
 'ACT6031_1',
 'ACT6033_1',
 'ACT6036_1',
 'ACT6038_1',
 'ACT6041_1',
 'ACT6042_1',
 'ACT6044_1',
 'ACT6047_1',
 'ACT6048_1',
 'ACT6049_1',
 'ACT6051_1',
 'ACT6052_1',
 'ACT6054_1',
 'ACT6058_1',
 'ACT6059_1',
 'ACT6062_1',
 'ACT6063_1',
 'ACT6064_1',
 'ACT6065_1',
 'ACT6066_1',
 'ACT6067_1',
 'ACT6069_1',
 'ACT6070_1',
 'ACT6074_1',
 'ACT6075_1',
 'ACT6076_1',
 'ACT6078_1',
 'ACT6080_1',
 'ACT6081_1',
 'ACT6083_1',
 'ACT6084_1',
 'ACT6085_1',
 'ACT6087_1',
 'ACT6089_1'

# Yeo pre-defined unthresholded metrics

In [ ]:
def add_community_sets(annotpwd,community_key):
    '''
    make a group annotation of a network parcellation into a list of sets
    
    returns a np.array list of communities from 
    Freesurfer annotations files for both hemispheres
    - note, for Freesurfer, index starts at 1 as 0 is medial wall but need 
    index to be 0 (so now network 1 is indexed as 0)
    '''
    def get_onehemi(hemi):
        #loads annotation integer labels (numpy array) for one hemi
        annot=fs.read_annot(annotpwd.format(hemi))

        community_labels=(pd.Series([x.astype(str).split('_')[2] 
                          for x in annot[2][1:]])
                .map(community_key))
        return community_labels.values

    hemiList=['lh','rh']
    net_labs=np.vstack([get_onehemi(hemi) 
                        for hemi in hemiList]).flatten()
    labs=pd.DataFrame({'parc':range(len(net_labs)),
                    'net':net_labs})
    community_sets=[set(x['parc'].values) 
                    for group,x in labs.groupby(['net'])]
    return community_sets

In [150]:
yeo_communities=add_community_sets(annotpwd,network7_key)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


In [157]:
#run and save node metrics with pre-defined Yeo parcellation
def get_unthresh_metrics_yeo(sub):
    starttime=time.time()
    funcdir=(funcpath+sub+'/'+ftype+'/average/_threshold_'+mscrub+'/'
    '_compcor_ncomponents_5_selector_pc10.'
    'linear1.wm1.'+nuis_global+'.motion1.quadratic1.gm0.compcor0.csf1/'
    '_bandpass_freqs_0.009.0.1/')
    subint=int(re.findall('\d+',sub)[0])
    edge=Edge(sub=subint,adjcsvdir=funcdir+adjcsv_name)
    edge.format_adj_df()
    edge_df_wcomms=merge_edges_communities(edge.edges_series,yeo_communities)
    node_metrics=calc_unthresh_metrics(edge_df_wcomms=edge_df_wcomms,
                          cost=None,sub=subint)
    print('Done with '+sub+'Time: '+str(time.time()-starttime))
    return node_metrics

In [158]:
df_nodes_yeo_unthresh=pd.concat(Parallel(8)(delayed(get_unthresh_metrics_yeo)
            (sub) for sub in subs_List))

Done with ACT08_1Time: 4.92322897911
Done with ACT07_1Time: 4.92374610901
Done with ACT01_1Time: 4.99646306038
Done with ACT12_1Time: 5.0005838871
Done with ACT04_1Time: 5.00756502151
Done with ACT09_1Time: 5.01227402687
Done with ACT05_1Time: 5.01751208305
Done with ACT11_1Time: 5.02140307426
Done with ACT13_1Time: 4.91984891891
Done with ACT16_1Time: 4.93030500412
Done with ACT18_1Time: 4.97618103027
Done with ACT20_1Time: 5.04828000069
Done with ACT22_1Time: 5.04312896729
Done with ACT23_1Time: 5.04029393196
Done with ACT24_1Time: 5.04429292679
Done with ACT21_1Time: 5.06719684601
Done with ACT25_1Time: 4.9527759552
Done with ACT6001_1Time: 4.95472693443
Done with ACT6002_1Time: 4.97664999962
Done with ACT6004_1Time: 4.95423197746
Done with ACT6005_1Time: 5.010627985
Done with ACT6011_1Time: 4.99843192101
Done with ACT6009_1Time: 5.01655912399
Done with ACT6006_1Time: 5.03986001015
Done with ACT6014_1Time: 4.88456702232
Done with ACT6015_1Time: 4.90848112106
Done with ACT6016_1Time:

# Format data

In [130]:
def load_data(sub,filename):
    funcdir=(funcpath+sub+'/'+ftype+'/average/_threshold_'+mscrub+'/'
             '_compcor_ncomponents_5_selector_pc10.'
             'linear1.wm1.'+nuis_global+'.motion1.quadratic1.gm0.compcor0.csf1/'
             '_bandpass_freqs_0.009.0.1/')
    return pd.read_csv(funcdir+filename)

In [131]:
def save_data(df,filename):
    funcdir=(gitpath+'data/'+ftype+'/average/_threshold_'+mscrub+'/'
             '_compcor_ncomponents_5_selector_pc10.'
             'linear1.wm1.'+nuis_global+'.motion1.quadratic1.gm0.compcor0.csf1/'
             '_bandpass_freqs_0.009.0.1/')
    return df.to_csv(funcdir+filename,index=False)

In [132]:
#load subject and node data
df_nodes=pd.concat(Parallel(5)(
    delayed(load_data)
    (sub=sub,filename='node_metrics_Louvain.csv')
    for sub in subs_List))

In [133]:
df_subs=pd.concat(Parallel(5)(
    delayed(load_data)
    (sub=sub,filename='sub_metrics_Louvain.csv')
    for sub in subs_List))

In [121]:
#load subject and node data
df_nodes_unthresh=pd.concat(Parallel(5)(
    delayed(load_data)
    (sub=sub,filename='node_metrics_Louvain_unthresh.csv')
    for sub in subs_List))

In [139]:
save_data(df_nodes,'node_metrics_Louvain.csv')

In [172]:
save_data(yeo_node_metrics_df,'node_metrics_Yeo_unthresh.csv')

In [140]:
save_data(df_subs.drop(['Unnamed: 0'],axis=1),'sub_metrics_Louvain.csv')

In [141]:
save_data(df_nodes_unthresh.drop(['Unnamed: 0'],axis=1),'node_metrics_Louvain_unthresh.csv')